In [ ]:
import numpy as np

In [ ]:
def mod_pert_random(low, likely, high, confidence=4, samples=30):
    """Produce random numbers according to the 'Modified PERT' 
    distribution. 
    
    :param low: The lowest value expected as possible.
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature 
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    confidence = min(8, confidence)
    confidence = max(2, confidence)

    mean = (low + confidence * likely + high)/(confidence + 2)

    a = (mean - low)/(high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)
    
    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta

In [1]:
class Queue:
    def __init__(self):
        self.groups=[]
    
    def queue_size(self):
        return len(self.groups)
    
    def isEmpty(self):
        if len(self.groups)> 0:
            return False
        else:
            return True
    
    def add_queue(self, group):
        if group.get_vip():   # If current group is a VIP group, move it forward by four groups,
            if len(self.groups) > 4:
                for i in range(0,4):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        break
                    else:
                        self.groups.insert(4,group)
            elif len(self.groups) > 1 and len(self.groups) < 4:
                for i in range(0,len(self.groups)):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        break
                    else:
                        self.groups.insert(1,group)
        else:
            self.groups.insert(0,group)
            
    
    def del_queue(self): #delete last=delete first come group 
        return self.groups.pop()  

IndentationError: expected an indented block (<ipython-input-1-173a5efcc011>, line 17)

In [7]:
# this class can be regarded as "class printer" in that case.
class Table:
    def __init__(self, num, size):
        self.num=num # No. of the table
        self.size = size # Size of the table: for group of up to 2, 4 or 6.
        self.currentGroup = None # Is the table occupied or not.
        self.time_remaining = 0 
        
    def busy(self):
        if self.currentGroup != None:
            return True
        else:
            return False      
    
    def startNext(self, newGroup):
        self.currentStatus = newGroup
    #    self.time_remaining = newtask.time_needs() # here maybe generate a random time from 10 - 60 mins using generator

In [ ]:
class Group:
    def __init__(self, time, size, vip):
        self.timestamp = time # Time when group registered (entered into the queue)
        self.size = size #randomly define size from 1 - 6
        self.vip = vip # Whether the group is a vip group
        self.table=None # Which table the group will be assigned to
        self.timeRequest = mod_pert_random(0, 45, 120, samples=1).astype(int)  #How long will the group spend
    
    def get_stamp(self):
        return self.timestamp
    
    def get_size(self):
        return self.size
    
    def wait_time(self, current_time):
        return current_time - self.timestamp
    
    def get_vip(self):
        return self.vip
    
    def get_time_request(self): 
        return self.timeRequest

In [ ]:
def tablesSetting(number_tables_2, number_tables_4, number_tables_6):
    table_2_list = []
    table_4_list = []
    table_6_list = [] 
    for i in range(number_tables_2):
        table_2_list.append(Table(i,2))
    
    for i in range(number_tables_4):
        table_4_list.append(Table(i+number_tables_2,4))
    
    for i in range(number_tables_6):
        table_6_list.append(Table(i+number_tables_4+number_tables_6,6))
    
    return (table_2_list, table_4_list, table_6_list)

In [2]:
def generation(Duration):
    size = np.random.randint(1,7,30)
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    table_2,table_4,table_6=tablesSetting(6,4,2)
    
    for i in range(Duration):
        #print(i,'/n')
        if i in timestamp_list:
            #print(size[counter],'**')
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter]))
                counter += 1
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter]))
                counter += 1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter]))
                counter += 1
                
        nextGroup_endTime={}
        
        if i in nextGroup_endTime_2.values():
            table_2[对应nextGroup_endTime_2值为i的keys].currentGroup = None  
            #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
            
        
        total_timeR = []
        
        for t in table_2:
            if (not t.busy()) and (not queue_2.isEmpty()):
                nextGroup = queue_2.del_queue()
                nextGroup_endTime_2[t] = i+nextGroup.timeRequest+2  #2min for table cleaning, rearragement and the time to walk in blabla
                total_timeR.append(nextGroup.timeRequest+2)
        
    avg_timeR = sum(total_timeR)/len(total_timeR)
        
    #return (queue_2, queue_4, queue_6)